In [1]:
import pandas
import numpy as onp
import jax.numpy as np

path = './data/kin8nm.csv'
data = pandas.read_csv(path, header=None).values

X_full = data[:, :-1]
Y_full = data[:, -1:]


N = X_full.shape[0]
n = int(N * 0.9)
ind = onp.arange(N)

onp.random.shuffle(ind)
train_ind = ind[:n]
test_ind = ind[n:]

X = X_full[train_ind]
Xs = X_full[test_ind]
Y = Y_full[train_ind]
Ys = Y_full[test_ind]

X_mean = np.mean(X, 0)
X_std = np.std(X, 0)
X = (X - X_mean) / X_std
Xs = (Xs - X_mean) / X_std
Y_mean = np.mean(Y, 0)
Y_std = np.std(Y, 0)
Y = (Y - Y_mean) / Y_std
Ys = (Ys - Y_mean) / Y_std

class D:
    X_train, Y_train, X_test, Y_test, X_mean, Y_mean, X_std, Y_std = X, Y, Xs, Ys, X_mean, Y_mean, X_std, Y_std
    D = X_train.shape[0]

from utils import bbData
d = bbData(D(), minibatch_size=128)

In [2]:
arg_dict = {'step_size': 0.001, \
            'sgd': False, 'alpha': 0.1, \
            'invsigma': 100., \
            'a': 1.0, 'mu': 10.0}


In [ ]:
import jax.numpy as np
import numpy as onp
from jax import random


from inference import Inference
from utils import bbData
from jax.scipy.special import logsumexp
from bnn_stax import *

key = random.PRNGKey(0)
k1, k2, k3 = random.split(key, 3)
sizes = [d.d.D] + [50] + [d.d.Y_test.shape[1]]
from jax.nn.initializers import normal, glorot_normal
model = BnnModel_stax_regression(d, n_layers=2, optim='sgd', stein_kernel='se', activation='relu')
width = 50
fd = model.func_dict
fd['LD_stein'] = LD_stein(**arg_dict)
fd['NHT_2_stein'] = NHT_2_stein(**arg_dict)
fd['HMC_stein'] = HMC_stein(**arg_dict)
model.stein_training(n_iter=5000, n_particles=20, key=k1, lr=arg_dict['step_size'], window=10, method=('NHT_2', 'stein'), sgd=arg_dict['sgd'], split=True)


In [5]:
print('Train log-likelihood: ' + "%.3f" % model.test_trace[-1][1])
print('Test log-likelihood: ' + "%.3f" % model.test_trace[-1][0])

Train log-likelihood: 0.19
Test log-likelihood: 0.60


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

# plot trace of test log-likelihood
plt.plot(np.linspace(0, 5000, len(model.test_trace)), [x[1] for x in model.test_trace])